In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1") #for paid openai embedding models dimensions to change for gemini mostly
)

In [14]:
import json
data=json.load(open("reviews.json"))
data["review"]


[{'professor': 'John Doe',
  'reviews': 'Professor Doe was incredibly knowledgeable and passionate about the subject. His lectures were engaging and easy to follow.',
  'subject': 'Introduction to Programming',
  'stars': 5},
 {'professor': 'Jane Smith',
  'review': 'The course material was challenging, but Professor Smith was always available to answer questions and provide extra help.',
  'subject': 'World History II',
  'stars': 4},
 {'professor': 'Michael Johnson',
  'review': "Professor Johnson's labs were very helpful in understanding the material. He was always patient and encouraging.",
  'subject': 'Human Anatomy and Physiology',
  'stars': 4.5},
 {'professor': 'Emily Davis',
  'review': 'Professor Davis was a wonderful mentor. She provided constructive feedback and helped me to develop my writing skills.',
  'subject': 'Creative Writing',
  'stars': 5},
 {'professor': 'David Wilson',
  'review': "The course was difficult, but Professor Wilson's lectures were clear and concise

In [16]:
processed_data = []
client=OpenAI()

for review in data["review"]:
    response = client.embeddings.create(
        input=review["review"],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["profesor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

KeyError: 'review'

In [ ]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

In [ ]:
index.describe_index_stats()